<a href="https://colab.research.google.com/github/phanikarra15/NLP_Final_Project/blob/main/Emphasis_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi


In [1]:
pip install transformers

     |████████████████████████████████| 1.3MB 2.8MB/s 
     |████████████████████████████████| 2.9MB 16.3MB/s 
     |████████████████████████████████| 890kB 28.5MB/s 
     |████████████████████████████████| 1.1MB 40.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=f74258c102301833bbb87429c5a88076813ab4230b63750b9e634b25ed5f3edb
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer, AutoModel


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
tokenizer = AutoTokenizer.from_pretrained("nghuyong/ernie-2.0-en")
print(device)

cpu


In [7]:
def dataset(filename):
    with open(filename,'r') as fp:
        lines = [line.strip() for line in fp]
    return lines

In [8]:
def wordData(data):
    wordLines = data
    words = []
    probabilities = []
    wordList = []
    pos = []
    empty = []
    for line in wordLines:
        lineSplit = line.strip().split('\t')
        if line:
            word = lineSplit[1]
            prob = lineSplit[4]
            temp = lineSplit[5]
            words.append(word)
            probabilities.append(prob)
            pos.append(temp)
        elif not (len(empty) and []):
            wordList.append((words, pos, probabilities))
            words = []
            probabilities = []
            pos = []
    return wordList

In [9]:
def wordtest(data):
    wordLines = data
    words = []
    testWord = []
    empty = []
    for line in wordLines:
        lineSplit = line.strip().split('\t')
        if line:
            word = lineSplit[1]            
            words.append(word)
        elif not len(empty):
            testWord.append(words)
            words = []       
    return testWord

In [10]:
def preTokenizing(data):
    t = []
    po = []
    prob = []
    for i,j,k in data:
        text = []
        pos = []
        probs = []
        for l in i:
            text.append(l)
        for m in j:
            pos.append(m)
        for n in k:
            probs.append(n)
        t.append(text)
        po.append(pos)
        prob.append(probs)
    return t,po, prob

In [11]:
def encoding(data):
    tokenized_text = []
    for i in data:
        sent = ""
        for j in i:
            if sent == "":
                sent += j
            else:
                sent = sent + " " + j
        tokens = tokenizer.tokenize(sent)
        tokenized_text.append(tokens)
        
    tokenized_text = tokenized_text[:10]
    
    # Encoding each word of the sentence and appending to a list
    en=[]
    for i in tokenized_text:
      en1 = []
      for j in i:
        en1.append(tokenizer.encode_plus(j, add_special_tokens=False, return_attention_mask=False, return_tensors='pt'))
      en.append(en1)
      en1 = []
      
    # Getting the input id's only for the list and passing them to the model 
    words = []  
    for d in en:
        words2 = []
        for i in d:
            words2.append(i['input_ids'])
        words.append(words2)
        words2 = []   
        
    return words

In [12]:
class BertBinaryClassifier(nn.Module):
    def __init__(self):
        super(BertBinaryClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained('nghuyong/ernie-2.0-large-en')
        self.linear = nn.Linear(1024, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, tokens):
        _, pooled_output = self.bert(tokens)
        linear_output = self.linear(pooled_output)
        proba = self.sigmoid(linear_output)
        proba = proba.data.tolist()
        return proba

In [13]:
TRAINING_FILE = "train.txt"
DEV_FILE = "dev.txt"
TEST_FILE = "test_data.txt"

In [14]:
trainText = wordData(dataset(TRAINING_FILE))
testEval = wordtest(dataset(TEST_FILE))
devText = wordData(dataset(DEV_FILE))

trainWords,trainTags, trainLabels = preTokenizing(trainText)
devWords, devTags, devLabels = preTokenizing(devText)

trainFinal = encoding(trainWords)
devFinal = encoding(devWords)

model = AutoModel.from_pretrained('nghuyong/ernie-2.0-en')

In [15]:
bert_clf = BertBinaryClassifier()
optimizer = torch.optim.Adam(bert_clf.parameters(), lr=3e-6)
bert_clf.train()

loss = nn.MSELoss()

In [16]:
token_answers = []
for epoch_num in range(10):
    print("epoch_num ---->{}".format(epoch_num))
    count = 0
    for batch_data, batch_probs in zip(trainFinal, trainLabels):
        bert_clf.zero_grad()
        count = count + 1
        answers_temp = []
        for i in batch_data:
            probas = bert_clf(i)
            for item in probas:
                for i in item:
                    answers_temp.append(i)            
        token_answers.append(answers_temp)
        print(answers_temp)
        answers_temp = []
        if count==5:
          break
        try:
          loss_func = nn.MSELoss(reduction = 'sum')
          print(answers_temp.size, batch_probs.size)
          batch_loss = loss_func(torch.tensor(answers_temp), torch.tensor(batch_probs))
          print("loss--->",batch_loss)
          batch_loss.backward()
          optimizer.step()
        except:
          continue

epoch_num ---->0
[0.4642156958580017, 0.39215847849845886, 0.5200430154800415]
[0.5311812162399292, 0.42274901270866394, 0.4341958463191986, 0.4749324321746826, 0.45046356320381165, 0.4826638102531433, 0.5772349834442139, 0.5682580471038818, 0.43465757369995117, 0.45676061511039734]
[0.5211614966392517, 0.46818074584007263, 0.45808297395706177, 0.5448566675186157, 0.5301490426063538, 0.5932053327560425, 0.4058903157711029, 0.5641112327575684, 0.5177786350250244]
[0.43581461906433105, 0.5322418808937073]
[0.490829735994339, 0.490474671125412, 0.5540384650230408, 0.5863052606582642, 0.38763338327407837, 0.4653547704219818, 0.5995450019836426, 0.5059840679168701, 0.5074765086174011, 0.4659624993801117, 0.4335924983024597, 0.5609588027000427]
epoch_num ---->1
[0.6077846884727478, 0.6156833171844482, 0.5463917851448059]
[0.5460423827171326, 0.4563975930213928, 0.45790204405784607, 0.4788731038570404, 0.36086615920066833, 0.4198920726776123, 0.40819236636161804, 0.45970144867897034, 0.476536